# Part II: Training a model

Date: 23/08/2022

Authors: Jordi Bolibar & Facundo Sapienza

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Since we have records of mass balance every 10 years, we will assign to each hydrological year in every period 2000-2010 and 2010-2020 the same part of mass balance that results from grabing the total mass balance in the 1o year period and divide it by 10. 

In [57]:
df_target = pd.read_csv('Data/mb_df.csv')

# Restrict to the two target peroids we have for training
df_target = df_target[df_target.period.isin(['2000-01-01_2010-01-01', '2010-01-01_2020-01-01'])]
# We add an unique id to each row to recognized repeited measurements 
df_target['target_id'] = np.arange(0, df_target.shape[0])

# Extract hydro year of the target 
dfs = []
for i in range(10):
    df_ = df_target.copy()
    df_['hydro_year'] = np.select([(df_.period=='2000-01-01_2010-01-01'),
                                         (df_.period=='2010-01-01_2020-01-01')], 
                                         [2000+i, 2010+i])
    dfs.append(df_)

df_target = pd.concat(dfs, axis=0)
df_target = df_target.set_index(['rgi_id', 'hydro_year'])
df_target = df_target.sort_values(['rgi_id', 'hydro_year'])
df_target = df_target[['dmdtda', 'err_dmdtda']]
df_target

dmdtda  err_dmdtda
rgi_id         hydro_year                      
RGI60-08.00001 2000       -0.458772     0.44912
               2001       -0.458772     0.44912
               2002       -0.458772     0.44912
               2003       -0.458772     0.44912
               2004       -0.458772     0.44912
...                             ...         ...
RGI60-08.03417 2015        0.389200     0.49660
               2016        0.389200     0.49660
               2017        0.389200     0.49660
               2018        0.389200     0.49660
               2019        0.389200     0.49660

[68340 rows x 2 columns]

In [58]:
df_features = pd.read_csv('Data/training_df.csv', index_col=['rgi_id', 'hydro_year'])
df_features

PDD          RGI_ID  area  icecap     lat  \
rgi_id         hydro_year                                                    
RGI60-08.00001 2000        40.322890  RGI60-08.00001  0.03       0  67.930   
               2001        42.489556  RGI60-08.00001  0.03       0  67.930   
               2002        38.044445  RGI60-08.00001  0.03       0  67.930   
               2003        39.844445  RGI60-08.00001  0.03       0  67.930   
               2004        40.577779  RGI60-08.00001  0.03       0  67.930   
...                              ...             ...   ...     ...     ...   
RGI60-08.03417 2015        51.437776  RGI60-08.03417  0.15       0  65.923   
               2016        55.643333  RGI60-08.03417  0.15       0  65.923   
               2017        50.003331  RGI60-08.03417  0.15       0  65.923   
               2018        57.004444  RGI60-08.03417  0.15       0  65.923   
               2019        54.643332  RGI60-08.03417  0.15       0  65.923   

                              prcp_01     prcp_02     prcp_03     prcp_04  \
rgi_id         hydro_year                                                   
RGI60-08.00001 2000         89.949440   39.799580   51.759113   50.980053   
               2001         50.515020   43.414460   49.670944   27.386324   
               2002         59.345170   29.263450   24.280972   44.494130   
               2003         48.070133   34.243984   27.780344   46.093716   
               2004         78.174040   31.993261   62.266920   28.106590   
...                               ...         ...         ...         ...   
RGI60-08.03417 2015        104.149030  111.409660  266.413970  155.734220   
               2016        217.098800  211.801880  242.586320  101.253310   
               2017         89.045910  149.197240  371.887760  299.347630   
               2018        176.113360  158.276730  192.674560   64.232956   
               2019        217.587550  137.044170  171.008670  167.764050   

                              prcp_05  ...    temp_06   temp_07   temp_08  \
rgi_id         hydro_year              ...                                  
RGI60-08.00001 2000         29.002270  ...  -6.611111 -1.732222  3.233333   
               2001         35.375156  ... -12.011110 -1.865555  1.533333   
               2002         44.540123  ...  -8.044444 -0.898889  3.333334   
               2003         26.073019  ...  -5.611111 -3.232222  4.133333   
               2004         40.665600  ...  -7.111110 -2.298889  3.033333   
...                               ...  ...        ...       ...       ...   
RGI60-08.03417 2015        189.870130  ...  -2.700000 -1.488889  2.322222   
               2016        108.876656  ...  -3.800000 -1.455556  4.088889   
               2017        121.731080  ...  -5.200000 -2.988889  1.355555   
               2018         24.806946  ...  -8.999999 -1.488889  5.588889   
               2019        227.375120  ...  -6.400000  0.211111  2.022222   

                             temp_09    temp_10    temp_11   temp_12    zmax  \
rgi_id         hydro_year                                                      
RGI60-08.00001 2000         9.202221  13.494446  10.450000  5.927779   250.0   
               2001        10.002221  13.594445  10.316667  7.627779   250.0   
               2002         9.202221  13.794446  10.316667  4.727778   250.0   
               2003         6.135554  15.094445  11.316667  6.494445   250.0   
               2004         8.102222  14.727779  11.216666  6.827778   250.0   
...                              ...        ...        ...       ...     ...   
RGI60-08.03417 2015         5.307778   8.482221  10.999999  6.890000  1318.0   
               2016         6.907778  10.382221   8.599999  7.290001  1318.0   
               2017         7.507778   9.482221   8.799999  6.890000  1318.0   
               2018         6.007778  12.582221   9.400000  5.990001  1318.0   
               2019         7.674444  10.182220   9.99

In [59]:
df_features.merge(df_target, 
                  left_on=['rgi_id','hydro_year'], 
                  right_on=['rgi_id','hydro_year'])

PDD          RGI_ID  area  icecap     lat  \
rgi_id         hydro_year                                                    
RGI60-08.00001 2000        40.322890  RGI60-08.00001  0.03       0  67.930   
               2001        42.489556  RGI60-08.00001  0.03       0  67.930   
               2002        38.044445  RGI60-08.00001  0.03       0  67.930   
               2003        39.844445  RGI60-08.00001  0.03       0  67.930   
               2004        40.577779  RGI60-08.00001  0.03       0  67.930   
...                              ...             ...   ...     ...     ...   
RGI60-08.03417 2015        51.437776  RGI60-08.03417  0.15       0  65.923   
               2016        55.643333  RGI60-08.03417  0.15       0  65.923   
               2017        50.003331  RGI60-08.03417  0.15       0  65.923   
               2018        57.004444  RGI60-08.03417  0.15       0  65.923   
               2019        54.643332  RGI60-08.03417  0.15       0  65.923   

                              prcp_01     prcp_02     prcp_03     prcp_04  \
rgi_id         hydro_year                                                   
RGI60-08.00001 2000         89.949440   39.799580   51.759113   50.980053   
               2001         50.515020   43.414460   49.670944   27.386324   
               2002         59.345170   29.263450   24.280972   44.494130   
               2003         48.070133   34.243984   27.780344   46.093716   
               2004         78.174040   31.993261   62.266920   28.106590   
...                               ...         ...         ...         ...   
RGI60-08.03417 2015        104.149030  111.409660  266.413970  155.734220   
               2016        217.098800  211.801880  242.586320  101.253310   
               2017         89.045910  149.197240  371.887760  299.347630   
               2018        176.113360  158.276730  192.674560   64.232956   
               2019        217.587550  137.044170  171.008670  167.764050   

                              prcp_05  ...   temp_08    temp_09    temp_10  \
rgi_id         hydro_year              ...                                   
RGI60-08.00001 2000         29.002270  ...  3.233333   9.202221  13.494446   
               2001         35.375156  ...  1.533333  10.002221  13.594445   
               2002         44.540123  ...  3.333334   9.202221  13.794446   
               2003         26.073019  ...  4.133333   6.135554  15.094445   
               2004         40.665600  ...  3.033333   8.102222  14.727779   
...                               ...  ...       ...        ...        ...   
RGI60-08.03417 2015        189.870130  ...  2.322222   5.307778   8.482221   
               2016        108.876656  ...  4.088889   6.907778  10.382221   
               2017        121.731080  ...  1.355555   7.507778   9.482221   
               2018         24.806946  ...  5.588889   6.007778  12.582221   
               2019        227.375120  ...  2.022222   7.674444  10.182220   

                             temp_11   temp_12    zmax    zmed    zmin  \
rgi_id         hydro_year                                                
RGI60-08.00001 2000        10.450000  5.927779   250.0   242.0   235.0   
               2001        10.316667  7.627779   250.0   242.0   235.0   
               2002        10.316667  4.727778   250.0   242.0   235.0   
               2003        11.316667  6.494445   250.0   242.0   235.0   
               2004        11.216666  6.827778   250.0   242.0   235.0   
...                              ...       ...     ...     ...     ...   
RGI60-08.03417 2015        10.999999  6.890000  1318.0  1186.0  1090.0   
               2016         8.599999  7.290001  1318.0  1186.0  1090.0   
               2017         8.799999  6.890000  1318.0  1186.0  1090.0   
               2018         9.400000  5.990001  1318.0  1186.0  1090.0   
               2019         9.999999  4.990001  1318.0  1186.0  1090.0   

                             dmdtda  e